# Merge data


In [94]:
import pandas as pd
from IPython.display import display
from torch import inf


month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# pre-processing exchange rate

exchange_rate: pd.DataFrame = pd.read_excel('data/macro-economic-indicators/Exchange-rate.xlsx')

exchange_rate['Year'] = exchange_rate['Year'].fillna(method='ffill')

exchange_rate['Year'] = exchange_rate['Year'].astype(str).apply(lambda x: x.split('/')[0])

exchange_rate['Month'] = exchange_rate['Month'].map(month_mapping)

exchange_rate = exchange_rate.dropna(subset=['Month'])

exchange_rate['Date'] = exchange_rate['Year'] + '-' + exchange_rate['Month'].astype(int).astype(str) + '-01'

exchange_rate['Date'] = pd.to_datetime(exchange_rate['Date'], format='%Y-%m-%d')

exchange_rate = exchange_rate[['Date', 'Rate']]
exchange_rate.rename(columns={'Rate': 'Exchange Rate'}, inplace=True)


# pre-processing gpd data
gdp_data = pd.read_excel('data/macro-economic-indicators/GDP.xlsx')


new_gdp_data = []
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for _, row in gdp_data.iterrows():
    year = int(row['Year'])
    for month in months:
        date = f"{year}-{int(month)}-01"
        new_row = [date] + row[1:].tolist()
        new_gdp_data.append(new_row)


new_gdp_data = pd.DataFrame(new_gdp_data, columns=['Date', 'GDP', 'Per Capita', 'Growth', 'Trade Balance (USD Billion)', '% of GDP', 'Contribution of Manufactoring Industry to GDP', 'No of People Going for Foreign Employment'])
new_gdp_data['Date'] = pd.to_datetime(new_gdp_data['Date'])

# merge new_gdp_data and exchange_rate
exchange_rate_gdp_data = pd.merge(exchange_rate, new_gdp_data, on='Date', how='left')


# pre-processing government data
government_revenue = pd.read_excel('data/macro-economic-indicators/Government-Revenue-and-Expenditure.xlsx')
government_revenue.fillna(government_revenue.mean(), inplace=True)
government_revenue['Date'] = pd.to_datetime(government_revenue['Date'])
# Extract year and month from 'Date' for grouping
government_revenue['YearMonth'] = government_revenue['Date'].dt.to_period('M')

# Group by Year-Month and sum 'Total Receipts of GON' and 'Total Expenditure'
monthly_data = government_revenue.groupby('YearMonth').agg(
    Monthly_Income=pd.NamedAgg(column='Total Receipts of GON', aggfunc='sum'),
    Monthly_Expenditure=pd.NamedAgg(column='Total Expenditure', aggfunc='sum')
).reset_index()

# Convert YearMonth to first date of the month
monthly_data['Date'] = monthly_data['YearMonth'].dt.start_time

government_revenue = monthly_data[['Date', 'Monthly_Income', 'Monthly_Expenditure']]

# merge exchange_rate_gdp_data and government_revenue
exchange_rate_gdp_govermment_data = pd.merge(government_revenue, exchange_rate_gdp_data, on='Date', how='left')
exchange_rate_gdp_govermment_data.head()


# inflation preprocessing
inflation = pd.read_excel('data/macro-economic-indicators/Inflation.xlsx')
inflation['Month'] = inflation['Month'].map(month_mapping)
inflation.dropna(inplace=True)
inflation['Date'] = inflation['Year'].astype(int).astype(str) + '-' + inflation['Month'].astype(int).astype(str) + '-01'
inflation['Date'] = pd.to_datetime(inflation['Date'], format='%Y-%m-%d')
inflation = inflation[['Date','Index','Change']]

# merge infaltion to above data
nepal_economy = exchange_rate_gdp_govermment_data.merge(inflation, on='Date', how='left')


# liquidity preprocessing

liquidity = pd.read_excel('data/macro-economic-indicators/Liquidity.xlsx')
liquidity['Date'] = pd.to_datetime(liquidity['Date'])

# Replace '-' with NaN and convert the columns to numeric
liquidity['Balance at NRB minus CRR'] = pd.to_numeric(
    liquidity['Balance at NRB minus CRR'].replace('-', pd.NA), errors='coerce')
liquidity['Prime Rate'] = pd.to_numeric(
    liquidity['Prime Rate'].replace('-', pd.NA), errors='coerce')
liquidity['Secondary Rate'] = pd.to_numeric(
    liquidity['Secondary Rate'].replace('-', pd.NA), errors='coerce')

# Extract Year-Month to group by month
liquidity.fillna(liquidity.mean(), inplace=True)
liquidity['YearMonth'] = liquidity['Date'].dt.to_period('M')

# Group by YearMonth and calculate monthly averages
liq_monthly_avg = liquidity.groupby('YearMonth').agg({
    'Balance at NRB minus CRR': 'mean',
    'Prime Rate': 'mean',
    'Secondary Rate': 'mean'
}).reset_index()

# Set Date as the first day of each month
liq_monthly_avg['Date'] = liq_monthly_avg['YearMonth'].dt.start_time

liq_monthly_avg = liq_monthly_avg[['Date', 'Balance at NRB minus CRR', 'Prime Rate', 'Secondary Rate']]
liq_monthly_avg.head()


#merge liquidity to nepal economy
nepal_economy = nepal_economy.merge(liq_monthly_avg, on='Date', how='left')


# preprocessing interest rate
interest_rate: pd.DataFrame = pd.read_excel('data/macro-economic-indicators/Nepal bank Base Interest Rate.xlsx')
#Melt the dataframe to convert months as rows
interest_rate = interest_rate.melt(id_vars=["FY"], var_name="Month", value_name="Interest Rate")

interest_rate['Date'] = pd.to_datetime(interest_rate['FY'].astype(str) + '-' + interest_rate['Month'] + '-01')

interest_rate = interest_rate[['Date', 'Interest Rate']]

# merge interest_rate to nepal economy
nepal_economy = nepal_economy.merge(interest_rate, on='Date', how='left')
nepal_economy.fillna(nepal_economy.mean(),inplace=True)

# use date as index

nepal_economy.set_index('Date', inplace=True)

nepal_economy.head()

/tmp/ipykernel_223113/1331525297.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  exchange_rate['Year'] = exchange_rate['Year'].fillna(method='ffill')


,Monthly_Income,Monthly_Expenditure,Exchange Rate,GDP,Per Capita,Growth,Trade Balance (USD Billion),% of GDP,Contribution of Manufactoring Industry to GDP,No of People Going for Foreign Employment,Index,Change,Balance at NRB minus CRR,Prime Rate,Secondary Rate,Interest Rate
Date,,,,,,,,,,,,,,,,
2021-07-01,283360.9,12701.9,125.087273,36.28883,1222.8788,4.2469,-12.117845,-33.3928,4.665598,50000.0,142.42,4.35,73177.185691,2.027138,2.795923,7.01
2021-08-01,3004465.2,400482.7,118.800000,36.28883,1222.8788,4.2469,-12.117845,-33.3928,4.665598,50000.0,142.40,3.49,27265.603242,3.477316,3.525055,6.83
2021-09-01,5382410.0,2358148.0,117.590000,36.28883,1222.8788,4.2469,-12.117845,-33.3928,4.665598,50000.0,143.42,4.24,22785.998293,4.684064,4.541698,6.81
2021-10-01,8406644.5,5903400.0,120.420000,36.28883,1222.8788,4.2469,-12.117845,-33.3928,4.665598,50000.0,146.54,6.04,29193.264593,4.456972,4.649000,6.70
2021-11-01,10677412.8,8726185.2,119.180000,36.28883,1222.8788,4.2469,-12.117845,-33.3928,4.665598,50000.0,146.09,7.11,18026.791357,4.533005,4.685647,6.38
